### 인공신경망 구현

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random

In [2]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다:", device)

다음 기기로 학습합니다: cuda


In [3]:
random.seed(777)
torch.manual_seed(777)
if device == "cuda":
  torch.cuda.manual_seed_all(777)

**Model code**

In [4]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()

    self.model = nn.Sequential(
        nn.Linear(784, 100),
        nn.ReLU(),
        nn.Linear(100, 100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )
  def forward(self, x):
    return self.model(x)

**Training code**

In [5]:
# hyperparameters
training_epochs = 3
batch_size = 100

# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                          train=False,
                          transform=transforms.ToTensor(),
                          download=True)

# dataset loader
data_loader = DataLoader(dataset=mnist_train,
                         batch_size = batch_size,
                         shuffle = True)

# MLP 객체 생성
model = MLP().to(device)
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

100%|██████████| 9912422/9912422 [00:00<00:00, 110870041.59it/s]


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 120413214.54it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31001983.14it/s]


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 15666553.26it/s]


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [6]:
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(data_loader)

  for idx, (x_train, y_train) in enumerate(data_loader):
    x_train = x_train.view(-1,28*28).to(device)
    y_train = y_train.to(device)

    outputs = model(x_train)
    cost = criterion(outputs, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('Epoch:', '%04d' % (epoch + 1), 'cosst =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cosst = 0.246218905
Epoch: 0002 cosst = 0.133334905
Epoch: 0003 cosst = 0.112655535
Learning finished


**Test code**

In [7]:
with torch.no_grad():
  x_test = mnist_test.data.view(-1,28*28).float().to(device)
  y_test = mnist_test.targets.to(device)

  prediction = model(x_test)
  correct_prediction = torch.argmax(prediction, 1) == y_test
  accuaracy = correct_prediction.float().mean()
  print("Accuracy:", accuaracy.item())

Accuracy: 0.9652999639511108


In [8]:
with torch.no_grad():
  wrong_indices = (correct_prediction == 0).nonzero(as_tuple=True)[0]
  r = random.randint(0,len(wrong_indices)-1)
  r = wrong_indices[r].cpu().item()
  x_single_data = mnist_test.data[r:r+1].view(-1,28*28).float().cuda()
  y_single_data = mnist_test.targets[r:r+1].cuda()